> `folium`을 활용해보자.

## 1. 라이브러리 imports

In [1]:
import numpy as np
import pandas as pd
#---#
import folium
import json
import requests

**[new !]** : `folium`, `json`, `requests`

## 2. Dictionary : view vs copy

\- 원하지 않는 코드

In [3]:
dct1 = {'a':1, 'b':2, 'c':3}
dct1

{'a': 1, 'b': 2, 'c': 3}

In [4]:
dct2 = dct1
dct1, dct2

({'a': 1, 'b': 2, 'c': 3}, {'a': 1, 'b': 2, 'c': 3})

In [6]:
dct2['c'] = 9999
dct1, dct2

({'a': 1, 'b': 2, 'c': 9999}, {'a': 1, 'b': 2, 'c': 9999})

> 둘다 바뀌어 버린다...? (아이디가 똑같기 때문임, 리소스가 저장되는 부분이 똑같음)

In [7]:
dct1 = {'a':1, 'b':2, 'c':3}
dct2 = dct1.copy()
dct2['c'] = 9999
dct1, dct2

({'a': 1, 'b': 2, 'c': 3}, {'a': 1, 'b': 2, 'c': 9999})

## 3. [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map)

\- 코로플레스 맵의 예시

![](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e1/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EC%A0%9C21%EB%8C%80_%EA%B5%AD%ED%9A%8C%EC%9D%98%EC%9B%90_%EC%84%A0%EA%B1%B0_%EA%B2%B0%EA%B3%BC.svg/1280px-%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EC%A0%9C21%EB%8C%80_%EA%B5%AD%ED%9A%8C%EC%9D%98%EC%9B%90_%EC%84%A0%EA%B1%B0_%EA%B2%B0%EA%B3%BC.svg.png)

> 대충 정의하면, coropleth = polygon + y라고 볼 수 있다.

* polygon(다각형 구역의 그림 및 큰 지도그림)은 지리적 구역을 표현
* y(색상)는 해당 지리적 구역에 대응하는 측정값

## 4. `folium` 기본

\- 개념
* Map Object를 생성(fig 생성)
* Map Object에 이것저것 추가(geom 추가)

### **A. `folium.Map()`**
---


`# 예시 1` : 기본

In [9]:
folium??

In [10]:
m = folium.Map()
m

> 세계지도를 불러온다. GIS가 탑재되어 있는듯.

In [15]:
m = folium.Map(
    scrollWheelZoom = False,
    location = [37, 127], zoom_start = 10
)
m

> `scrollWheelZoom`을 `False`로 설정하여 문서를 보다 스크롤로 줌인-줌아웃 되지 않도록 바꿔줌.
>
> `location`을 지정하여 위도와 경도 설정 가능
>
> `zoom_start`를 지정하여 확대 단계를 지정 가능

\- 좌표 정보는 구글맵스에서 지역을 더블클릭하여 따올 수 있다.

35.846737, 127.129374

In [19]:
folium.Map(
    scrollWheelZoom = False,
    location = [35.846737, 127.129374], zoom_start = 18
)

> 18이 최대값이고 그 이상 값은 18로 설정된다.

### **B. `folium.Marker()`**
---




`# 예제1` : `Map`에 `Marker`를 추가


In [20]:
m = folium.Map(
    scrollWheelZoom = False,
    location = [35.846737, 127.129374], zoom_start = 18
)

In [23]:
marker = folium.Marker(location = [35.846737, 127.129374])

In [24]:
m

> 아직은 마커가 `Map`인스턴스에 표시되지 않음.

In [25]:
marker.add_to(m)

In [26]:
m

> `Marker`에서의 메소드로 추가하였음.

`# 예제 2` - 통계학과 대학원생의 산책경로

In [ ]:
## 통계학과 대학원생의 산책경로..
[35.8471, 127.1291]
[35.8468, 127.1289]
[35.84635, 127.1291]
[35.84635, 127.1297]
[35.8468, 127.12995]
[35.8474, 127.1300]

In [28]:
m = folium.Map(
    scrollWheelZoom = False,
    location = [35.8468,127.1294],  ## 분수대
    zoom_start = 18
)

folium.Marker(location = [35.8471, 127.1291]).add_to(m)
folium.Marker(location = [35.8468, 127.1289]).add_to(m)
folium.Marker(location = [35.84635, 127.1291]).add_to(m)
folium.Marker(location = [35.84635, 127.1297]).add_to(m)
folium.Marker(location = [35.8468, 127.12995]).add_to(m)
folium.Marker(location = [35.8474, 127.1300]).add_to(m)
m

\- 한번에...

In [29]:
m = folium.Map(
    scrollWheelZoom = False,
    location = [35.8468,127.1294],  ## 분수대
    zoom_start = 18
)

folium.Marker(location = [[35.8471, 127.1291],
  [35.8468, 127.1289],
  [35.84635, 127.1291],
  [35.84635, 127.1297],
  [35.8468, 127.12995],
  [35.8474, 127.1300]]
).add_to(m)

ValueError: ignored

> 이런건 안됨

### **C. `folium.Polygon`**
---




`# 예제 1` - 산책경로를 폴리곤으로 표시(더블리스트)

In [31]:
m = folium.Map(
    scrollWheelZoom=False,
    location = [35.8468,127.1294], # 분수대
    zoom_start=18,
)
folium.Polygon(
    locations = [[35.8471, 127.1291],
                 [35.8468, 127.1289],
                 [35.84635, 127.1291],
                 [35.84635, 127.1297],
                 [35.8468, 127.12995],
                 [35.8474, 127.1300]],
    fill=True   ## 속까지 채워줌
).add_to(m)
m

`# 예제 2` - 2개의 폴리곤으로 표시(3중 리스트)

In [33]:
poly = np.array([[35.8471, 127.1291],
                 [35.8468, 127.1289],
                 [35.84635, 127.1291],
                 [35.84635, 127.1297],
                 [35.8468, 127.12995],
                 [35.8474, 127.1300]])

In [34]:
lat, lon = poly.T

In [36]:
poly2 = np.stack([lat, lon + 0.0011], axis = 1)  ## 경도에다가 0.0011을 더해줬음, concatenate와 달리 이것은 차원을 늘려줌
poly2

array([[ 35.8471 , 127.1302 ],
       [ 35.8468 , 127.13   ],
       [ 35.84635, 127.1302 ],
       [ 35.84635, 127.1308 ],
       [ 35.8468 , 127.13105],
       [ 35.8474 , 127.1311 ]])

In [40]:
np.stack([poly, poly2], axis = 0)  ## 두 폴리곤을 삼중 리스트 형태로 변환

array([[[ 35.8471 , 127.1291 ],
        [ 35.8468 , 127.1289 ],
        [ 35.84635, 127.1291 ],
        [ 35.84635, 127.1297 ],
        [ 35.8468 , 127.12995],
        [ 35.8474 , 127.13   ]],

       [[ 35.8471 , 127.1302 ],
        [ 35.8468 , 127.13   ],
        [ 35.84635, 127.1302 ],
        [ 35.84635, 127.1308 ],
        [ 35.8468 , 127.13105],
        [ 35.8474 , 127.1311 ]]])

In [46]:
m = folium.Map(
    scrollWheelZoom=False,
    location = [35.8468,127.1294], # 분수대
    zoom_start=18,
)
folium.Polygon(
    locations = np.stack([poly, poly2], axis = 0),   ## 점이 여러개니까 s 붙여줌
    fill=True
).add_to(m)
m

> 아이디어 : 이걸 잘 이용하면 코로플레스맵처럼 지도를 행정구역별로 나눌 수 있겠음. 근데 실제 구현하려면 엄청난 노가다가 필요할 것 같음... -> 누군가 해놓지 않았을까???

## 5. South Korea github

### **A. github 소개**
---




* `southkorea`라는 깃허브 유저가 있는데, 이곳엔 `southkorea-maps`라는 저장소가 있다.

* 저장소 `https://github.com/southkorea/southkorea-maps`에는 `kostat/2018/json/`이란 폴더가 있고, 아래의 파일들이 있음.

```
skorea-municipalities-2018-geo.json # <-- 이 파일에 관심있음.
skorea-municipalities-2018-topo-simple.json
skorea-municipalities-2018-topo.json
skorea-provinces-2018-geo.json # <-- 이 파일에 관심있음.
skorea-provinces-2018-topo-simple.json
skorea-provinces-2018-topo.json
skorea-submunicipalities-2018-geo.json
skorea-submunicipalities-2018-topo-simple.json
skorea-submunicipalities-2018-topo.json
```

이중 관심있는 아래의 두 파일

```
skorea-municipalities-2018-geo.json
skorea-provinces-2018-geo.json
```

> 이게 행정구역을 의미하는 폴리곤을 노가다로 정리해 둔 것임.

### **B. json 파일 다운로드**
---




In [47]:
global_dict = json.loads(requests.get('https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2018/json/skorea-provinces-2018-geo.json').text)
local_dict = json.loads(requests.get('https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2018/json/skorea-municipalities-2018-geo.json').text)

In [48]:
global_dict.keys()

dict_keys(['type', 'features', 'name', 'crs'])

In [49]:
local_dict.keys()

dict_keys(['type', 'features', 'name', 'crs'])

In [53]:
global_dict['type']

'FeatureCollection'

In [54]:
global_dict['name']

'sido'

In [55]:
global_dict['crs']

{'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}}

### **C. json 파일의 구조**
---




In [56]:
global_dict['features']

Output hidden; open in https://colab.research.google.com to view.

In [57]:
global_dict['features'][0]  ## 딕셔너리임

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[127.02214829071995, 37.6997208220174],
    [127.02531549179129, 37.69958052666555],
    [127.0268980715665, 37.700251138801015],
    [127.02702219174589, 37.70111540651664],
    [127.02768734803148, 37.700937381678756],
    [127.02899106854076, 37.69965827866671],
    [127.02928557925306, 37.69929162349923],
    [127.02960141563474, 37.69817519597225],
    [127.02965442206329, 37.69780663241529],
    [127.02972931239567, 37.69627243691011],
    [127.03016631976621, 37.69519820573226],
    [127.03097248066267, 37.693556837990705],
    [127.0324142885382, 37.691839038874726],
    [127.03577652734599, 37.69237420164817],
    [127.03676285064714, 37.69263992022006],
    [127.03790670521317, 37.69327193541787],
    [127.03999748131439, 37.69468439213186],
    [127.04110878733236, 37.69529978284375],
    [127.0418126234525, 37.69538228560054],
    [127.04307253543809, 37.69523090725608],
    [127.04481711632289, 37.6929

In [58]:
global_dict['features'][0]['geometry']  ## 또 딕셔너리임

{'type': 'Polygon',
 'coordinates': [[[127.02214829071995, 37.6997208220174],
   [127.02531549179129, 37.69958052666555],
   [127.0268980715665, 37.700251138801015],
   [127.02702219174589, 37.70111540651664],
   [127.02768734803148, 37.700937381678756],
   [127.02899106854076, 37.69965827866671],
   [127.02928557925306, 37.69929162349923],
   [127.02960141563474, 37.69817519597225],
   [127.02965442206329, 37.69780663241529],
   [127.02972931239567, 37.69627243691011],
   [127.03016631976621, 37.69519820573226],
   [127.03097248066267, 37.693556837990705],
   [127.0324142885382, 37.691839038874726],
   [127.03577652734599, 37.69237420164817],
   [127.03676285064714, 37.69263992022006],
   [127.03790670521317, 37.69327193541787],
   [127.03999748131439, 37.69468439213186],
   [127.04110878733236, 37.69529978284375],
   [127.0418126234525, 37.69538228560054],
   [127.04307253543809, 37.69523090725608],
   [127.04481711632289, 37.6929362514031],
   [127.045094291979, 37.692385199610435],

In [60]:
global_dict['features'][0]['geometry']['coordinates']  ## 우리가 원하는 3중 리스트

[[[127.02214829071995, 37.6997208220174],
  [127.02531549179129, 37.69958052666555],
  [127.0268980715665, 37.700251138801015],
  [127.02702219174589, 37.70111540651664],
  [127.02768734803148, 37.700937381678756],
  [127.02899106854076, 37.69965827866671],
  [127.02928557925306, 37.69929162349923],
  [127.02960141563474, 37.69817519597225],
  [127.02965442206329, 37.69780663241529],
  [127.02972931239567, 37.69627243691011],
  [127.03016631976621, 37.69519820573226],
  [127.03097248066267, 37.693556837990705],
  [127.0324142885382, 37.691839038874726],
  [127.03577652734599, 37.69237420164817],
  [127.03676285064714, 37.69263992022006],
  [127.03790670521317, 37.69327193541787],
  [127.03999748131439, 37.69468439213186],
  [127.04110878733236, 37.69529978284375],
  [127.0418126234525, 37.69538228560054],
  [127.04307253543809, 37.69523090725608],
  [127.04481711632289, 37.6929362514031],
  [127.045094291979, 37.692385199610435],
  [127.04862943554828, 37.69406226137355],
  [127.048807

In [59]:
# local_dict

In [75]:
np.array(global_dict['features'][0]['geometry']['coordinates']).shape

(1, 1498, 2)

> 서울의 다각형을 의미하는 것 같음(확인해보자)

In [83]:
np.array(global_dict['features'][0]['geometry']['coordinates']).shape  ## 쓸데없이 3차원

(1, 1498, 2)

In [73]:
np.array(global_dict['features'][0]['geometry']['coordinates'])[0,:,:].shape  ## 경도와 위도, 2차원

(1498, 2)

In [82]:
np.array(global_dict['features'][0]['geometry']['coordinates'])[0,:,1].shape  ## 위도, 1차원

(1498,)

In [81]:
np.array(global_dict['features'][0]['geometry']['coordinates'])[0, :, 0].shape  ## 경도, 1차원

(1498,)

In [70]:
lon, lat = np.array(global_dict['features'][0]['geometry']['coordinates'])[0,:,:].T

In [85]:
np.stack([lat, lon], axis = 1).shape

(1498, 2)

> 아까는 위도 경도로 저장되었는데, 이제 경도 위도로 저장이 되었음. → 바꿔줌

\- 이게 어떤 원리냐면...

* [0,:,:] 첫번째 원소만 선택 > 멀티 리스트를 깨줌
* T > 전치시켜서 두 개의 리스트로 만듦
* lon, lat에 각자 저장함(순서를 바꾸기 위함)
* np.stack([lat, lon], axis = 1) > 열로 스택을 쌓아줌(2차원으로 변경)

In [87]:
m = folium.Map(
    location = [37.55,127],
    zoom_start=11,
    scrollWheelZoom = False
)
folium.Polygon(
    locations = np.stack([lat,lon],axis=1).tolist(),    ## tolist()는 안해도 됨
    fill = True
).add_to(m)
m

> 일상언어에서는 (lat,lon) 순서로 표현한다. 즉 적도를 기준으로 얼마나 위/아래로 있는지, 그리고 그리니치천문대를 기준으로 얼마나 동/서로 있는지를 표현한다. 그런데 lat이 y축의 느낌을 가지고 lon이 x축의 느낌을 가지는데 (lat,lon) 순으로 좌표를 선택하면 컴퓨터로 표현하기에 종종 헷갈릴 수 있다. 그래서 어떤 경우는 (lon,lat) 순서로 좌표를 표현하기도 한다. > 경도, 위도 순

## 6. `folium.Choropleth`를 이용한 시각화

### **A. `folium.Choropleth` 소개**
---




\- `folium.Choropleth`는 아래와 같은 방식으로 그림을 그린다고 생각하면 편리하다.

1. `json` 파일을 바탕으로 폴리곤을 그린다. 폴리곤에 이름을 붙인다.
2. `df = [폴리곤의 이름, 통계값(y)]와 같은 형식으로 정리된 데이터프레임을 바탕으로, 각 폴리곤에 대응하는 `y`값을 색깔로 매핑한다.

```
서울 - 's' -- 얼만큼 빨간색으로 할지
경기 - 'g' -- 얼만큼 파란색으로 할지
```

```
df
지역 | 값
s | 10
g | 20
```

> 각 폴리곤의 이름과 통계값들이 정리가 되어있어야 함.

### **B. Polygon 시각화**
---




`# 예제1` : 전국의 행정구역 시각화(global)

In [89]:
m = folium.Map(
    location = [36, 128],
    zoom_start = 7,
    scrollWheelZoom = False
)

folium.Choropleth(
    geo_data = global_dict
).add_to(m)
m

Output hidden; open in https://colab.research.google.com to view.

> 그대로 넣어버리면 바로 해준다.

`# 예제2` : 전국의 행정구역 시각화(local)

In [90]:
m = folium.Map(
    location = [36, 128],
    zoom_start = 7,
    scrollWheelZoom = False
)
folium.Choropleth(
    geo_data = local_dict
).add_to(m)
m

Output hidden; open in https://colab.research.google.com to view.

> 폴리곤을 그릴 수 있는 딕셔너리 타입의 정보가 들어간다.

`# 예제3` : 전국의 행정구역 시각화(덕진구/완산구)

In [91]:
local_jeonju = local_dict.copy()

In [ ]:
local_jeonju

In [94]:
local_dict['features'][15]['properties']  ## features의 properties에는 지역 이름에 대한 정보가 포함됨

{'name': '강서구', 'base_year': '2018', 'name_eng': 'Gangseo-gu', 'code': '11160'}

In [96]:
len(local_dict['features'])

250

In [99]:
[local_dict['features'][i] for i in range(250) if local_dict['features'][i]['properties']['name'] == '전주시덕진구' or local_dict['features'][i]['properties']['name'] == '전주시완산구']
## 아래의 코드가 더 명확하긴 하다.
#[l for l in local_dict['features'] if l['properties']['name'] == '전주시덕진구' or l['properties']['name'] == '전주시완산구']

[{'type': 'Feature',
  'geometry': {'type': 'MultiPolygon',
   'coordinates': [[[[127.11044780083574, 35.838949832816176],
      [127.1135948524044, 35.837854486310746],
      [127.1139393425632, 35.837848233581205],
      [127.11435513472738, 35.83784184083071],
      [127.11518661050826, 35.83836645128747],
      [127.11515332831482, 35.83838977941879],
      [127.11523061589928, 35.83843384975816],
      [127.11591558913986, 35.83849511540428],
      [127.11649941545609, 35.8385449645881],
      [127.11671724552282, 35.838562492613725],
      [127.1169183684167, 35.83857831613743],
      [127.1170210991372, 35.83858252410064],
      [127.11719355090722, 35.83858623369339],
      [127.11754826532164, 35.838569128848576],
      [127.11782249206348, 35.83853668936038],
      [127.11802163492057, 35.838476833317145],
      [127.11822440990959, 35.83840345527894],
      [127.11847743016729, 35.83825242907003],
      [127.12093095186137, 35.83586689829406],
      [127.12274668286028, 35.8

> properties의 name값에는 전주시덕진구가 들어감

In [100]:
local_jeonju['features'] = [local_dict['features'][i] for i in range(250) if local_dict['features'][i]['properties']['name'] == '전주시덕진구' or local_dict['features'][i]['properties']['name'] == '전주시완산구']

In [101]:
m = folium.Map(
    location = [35.84195368311022, 127.1155556693179],
    zoom_start = 11,
    scrollWheelZoom = False
)

folium.Choropleth(
    geo_data = local_jeonju
).add_to(m)

m

> geo_data에 포함되어야 할 정보를 모두 가지기 위해 features만 변환한 모습

### **C. (Polygon, Value) 시각화(★★★)**
---




`# 예제1` : 덕진구 vs 완산구

덕진구와 완산구의 전기 사용량이 아래와 같이 정리되었다고 하자.

In [102]:
df = pd.DataFrame({
    'key':['전주시덕진구', '전주시완산구'],
    'elec_use':[20,30]
})
df

,key,elec_use
0,전주시덕진구,20
1,전주시완산구,30


(선실습 후이해)

In [103]:
m = folium.Map(
    location = [35.84195368311022, 127.1155556693179],
    zoom_start=11,
    scrollWheelZoom = False
)
folium.Choropleth(
    geo_data=local_jeonju,    ## Json파일로 폴리곤을 그린다.
    key_on='properties.name',   ## ['properties']['name'] : 한글이름임, 폴리곤에 이름을 붙인다. 붙이지 않으면 key와 연결되지 못한다.
    data=df, ## 네임과 데이터 정보
    columns=['key','elec_use']  ## 해당하는 네임과 y값
).add_to(m)
m

> 두 개의 정보를 이용한다.

* 이해를 위해 필요한 약관의 직관
  * 코로플레스맵을 그리기 위해서는 항상 두개의 데이터(geo_data, data)를 연결해야 하는 구조이다.
  * 두 개의 데이터를 연결하기 위해서는 공유가능한 연결의 매개체가 필요하다.
  * 코로플레스맵의 연결매개체는 '전주시완산구', '전주시덕진구'와 같은 지역명이다.

`# 예제2` : 덕진구 vs 완산구 / `key_on`에 대한 이해를 위해 만든 억지예제

덕진구와 완산구 전기 사용량이 아래와 같이 정리되었다고 하자.

In [104]:
df = pd.DataFrame({
    'key':['전주시덕진구', 'Jeonjusiwansangu'],
    'elec_use':[20,30]
})
df

,key,elec_use
0,전주시덕진구,20
1,Jeonjusiwansangu,30


In [105]:
m = folium.Map(
    location = [35.84195368311022, 127.1155556693179],
    zoom_start=11,
    scrollWheelZoom = False
)

folium.Choropleth(
    geo_data = local_jeonju,
    key_on = 'properties.name',   ## 한글로 된 이름만 포함함
    data = df,
    columns = ['key', 'elec_use']  ## 순서가 뒤바뀌면 안됨
).add_to(m)

m

> 깨짐

In [106]:
m = folium.Map(
    location = [35.84195368311022, 127.1155556693179],
    zoom_start=11,
    scrollWheelZoom = False
)
folium.Choropleth(
    geo_data=local_jeonju,
    key_on='properties.name_eng',
    data=df,
    columns=['key','elec_use']
).add_to(m)
m

> 이번엔 반대로 깨짐(한글 이름은 name_eng에는 없으니까...)

\- 예시2 : `code`열을 새롭게 할당하고 'code'열로 `key_on`

In [107]:
df.assign(code = ['35012','30511'])

,key,elec_use,code
0,전주시덕진구,20,35012
1,Jeonjusiwansangu,30,30511


In [108]:
m = folium.Map(
    location = [35.84195368311022, 127.1155556693179],
    zoom_start=11,
    scrollWheelZoom = False
)
folium.Choropleth(
    geo_data=local_jeonju,
    key_on='properties.code',
    data=df.assign(code = ['35012','35011']),
    columns=['code','elec_use']
).add_to(m)
m

> 코드로 하면 확실하게 가져올 수 있다. (문자 저장에 따른 문제를 해결할 수 있음)

`# 예제 4` : 대한민국 인구수 시각화

In [109]:
df=pd.read_csv('https://raw.githubusercontent.com/guebin/2021DV/master/_notebooks/2021-11-22-prov.csv')
df

,행정구역(시군구)별,총인구수 (명)
0,서울특별시,9532428
1,부산광역시,3356311
2,대구광역시,2390721
3,인천광역시,2945009
4,광주광역시,1442454
5,대전광역시,1454228
6,울산광역시,1122566
7,세종특별자치시,368276
8,경기도,13549577
9,강원도,1537717


In [113]:
global_dict['features'][0]['properties']

{'name': '서울특별시', 'base_year': '2018', 'name_eng': 'Seoul', 'code': '11'}

> 이런 식으로 저장되어 있음, 바꿀 건 따로 없어보이니까...

In [114]:
m = folium.Map(
    location = [36,128],
    zoom_start=7,
    scrollWheelZoom = False
)
folium.Choropleth(
    geo_data=global_dict,
    key_on='properties.name',
    data=df,
    columns=['행정구역(시군구)별','총인구수 (명)']    ## 바로 넣음
).add_to(m)
m

Output hidden; open in https://colab.research.google.com to view.

`# 예제 5` : 대한민국 인수구 시각화(local)

In [118]:
df=pd.read_csv('https://raw.githubusercontent.com/guebin/2021DV/master/_notebooks/2021-11-22-muni.csv')
df

,행정구역(시군구)별,총인구수 (명)
0,종로구,145346
1,중구,122781
2,용산구,223713
3,성동구,287174
4,광진구,340814
...,...,...
269,함양군,38475
270,거창군,61242
271,합천군,43029
272,제주시,493225


In [121]:
m = folium.Map(
    scrollWheelZoom = False,
    zoom_start = 7,
    location = [36, 128]
)

folium.Choropleth(
    geo_data = local_dict,
    key_on = 'properties.name',
    data = df,
    columns = ['행정구역(시군구)별', '총인구수 (명)']
).add_to(m)

m

> 이름의 입력이 고르지 않아 깨진 부분이 있다... 코드로 바꿔야 할 것.